# Lesson 4 — Robot Animation (Personality)

Goal:
- give the robot character
- use **camera moves + eyes** (fast-sdk)
- use **speech** via `tts_lib.py`

No navigation. No sensors.


In [ ]:
# --- Bootstrap (same in every notebook) ---
from pathlib import Path
import sys

def add_repo_root():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            if str(p) not in sys.path:
                sys.path.insert(0, str(p))
            print('Repo root:', p)
            return p
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_root()

In [ ]:
import time, random, importlib

# Your TTS lib (in common/lib on the robot image)
import tts_lib as tts
importlib.reload(tts)
tts.warm_piper('ryan')

# fast-sdk camera + eyes
try:
    from fast_hi_wonder import Camera
    cam = Camera()
    print('✔ fast-sdk Camera')
except Exception as e:
    cam = None
    print('✖ fast-sdk Camera not available ->', e)

try:
    from fast_hi_wonder import RGB
    eyes_hw = RGB()
    print('✔ fast-sdk RGB (eyes)')
except Exception as e:
    eyes_hw = None
    print('✖ fast-sdk RGB not available ->', e)

if cam is None:
    raise RuntimeError('Lesson 4 expects fast-sdk Camera. Install/verify turbopi-fast-sdk on this robot image.')


In [ ]:
def say(text):
    path = tts.pre_synth(text, voice='ryan', length_scale='0.98', sentence_silence='0.08')
    tts.play_path_async(path)

def set_eyes(color):
    if not eyes_hw:
        return
    try:
        if hasattr(eyes_hw, 'set_all'):
            eyes_hw.set_all(color)
        elif hasattr(eyes_hw, 'set'):
            eyes_hw.set(0, color); eyes_hw.set(1, color)
    except Exception:
        pass

def cam_call(name, *a, **k):
    if hasattr(cam, name):
        return getattr(cam, name)(*a, **k)
    return None

print('✔ helpers ready')

## Build some expressions

These are your animation building blocks.


In [ ]:
def happy():
    set_eyes((0,255,80))
    say('Kia ora! I am happy!')
    cam_call('wiggle', cycles=2, amplitude=160)
    cam_call('nod')

def curious():
    set_eyes((120,120,255))
    say('Hmm. What is that?')
    cam_call('glance_left')
    cam_call('glance_right')
    cam_call('center_all')

def surprised():
    set_eyes((255,220,0))
    say('Oh!')
    cam_call('look_up')
    cam_call('shake')

def sleepy():
    set_eyes((30,30,80))
    say('I am a bit sleepy.')
    cam_call('look_down')
    cam_call('tiny_wiggle', seconds=1.2, amplitude=70)

print('✔ expressions ready')

In [ ]:
# Try each one
happy(); time.sleep(0.4)
curious(); time.sleep(0.4)
surprised(); time.sleep(0.4)
sleepy(); time.sleep(0.4)
cam_call('center_all')
set_eyes((0,0,0))

## Personality loop

Run a simple mood machine for 20 seconds.


In [ ]:
MOODS = [happy, curious, surprised, sleepy]

def personality(seconds=20):
    end = time.time() + float(seconds)
    say('Watch my personality.')
    while time.time() < end:
        random.choice(MOODS)()
        time.sleep(0.3)
    cam_call('center_all')
    set_eyes((0,0,0))

personality(20)